# Imaging demonstration

### This script works through the high level arl interface to crocodile, making a fake data set and then deconvolving it. Finally the full and residual visibility are plotted.

In [ ]:
%matplotlib inline

import sys, os
sys.path.append(os.path.join('..','..'))

from matplotlib import pylab
pylab.rcParams['figure.figsize'] = (12.0, 12.0)
pylab.rcParams['image.cmap'] = 'rainbow'

import numpy

from astropy.coordinates import SkyCoord
from astropy.wcs.utils import skycoord_to_pixel, pixel_to_skycoord
from astropy import units as u
from astropy import constants as const

from matplotlib import pyplot as plt


from arl.image.deconvolution import deconvolve_cube
from arl.visibility.operations import create_visibility
from arl.skymodel.operations import create_skymodel_from_image, add_component_to_skymodel, create_skycomponent, \
    find_skycomponent
from arl.image.operations import show_image, import_image_from_fits, export_image_to_fits
from arl.util.testing_support import create_named_configuration, replicate_image, create_test_image
from arl.fourier_transforms.ftprocessor import predict_2d, invert_2d, create_image_from_visibility


import logging
log = logging.getLogger()
log.setLevel(logging.DEBUG)
log.addHandler(logging.StreamHandler(sys.stdout))

### Construct a VLA configuration and then shrink it to match our test image.

In [ ]:
parameters = {}

vlaa = create_named_configuration('VLAA')
vlaa.data['xyz']=vlaa.data['xyz']/10.0

### We create the visibility. This just makes the uvw, time, antenna1, antenna2, weight columns in a table

In [ ]:
times = numpy.arange(-numpy.pi/2.0, +numpy.pi/2.0,0.05)
frequency = numpy.array([1e8])

reffrequency = numpy.max(frequency)
phasecentre = SkyCoord(0.0*u.rad, u.rad*numpy.pi/4, frame='icrs', equinox=2000.0)
vt = create_visibility(vlaa, times, frequency, weight=1.0, phasecentre=phasecentre)

### Plot the synthesized uv coverage, including for MFS

In [ ]:
plt.clf()
for f in frequency:
    x=f/const.c
    plt.plot(x*vt.data['uvw'][:,0], x*vt.data['uvw'][:,1], '.', color='b')
    plt.plot(-x*vt.data['uvw'][:,0], -x*vt.data['uvw'][:,1], '.', color='r')

### Read the venerable test image, constructing an image

In [ ]:
parameters={}
parameters['npixel']=512
parameters['cellsize']=0.0001
parameters['wstep']=30.0

m31image = create_test_image(npol=4, nchan=len(frequency), cellsize=parameters['cellsize'])
fig=show_image(m31image)

In [ ]:
vt = predict_2d(vt, m31image, params=parameters)

# To check that we got the prediction right, plot the amplitude of the visibility.
uvdist=numpy.sqrt(vt.data['uvw'][:,0]**2+vt.data['uvw'][:,1]**2)
plt.clf()
plt.plot(uvdist, numpy.abs(vt.data['vis'][:,0,0]), '.')
plt.xlabel('uvdist')
plt.ylabel('Amp Visibility')
plt.show()

### Make the dirty image and point spread function

In [ ]:
model = create_image_from_visibility(vt, params=parameters)
dirty = invert_2d(vt, model, params=parameters)
psf = invert_2d(vt, model, dopsf=True, params=parameters)
psfmax = psf.data.max()
dirty.data = dirty.data/psfmax
psf.data = psf.data/psfmax

show_image(dirty)
print("Max, min in dirty image = %.6f, %.6f, psfmax = %f" % (dirty.data.max(), dirty.data.min(), psfmax))

print("Max, min in PSF         = %.6f, %.6f, psfmax = %f" % (psf.data.max(), psf.data.min(), psfmax))

export_image_to_fits(dirty, 'imaging_dirty.fits')
export_image_to_fits(psf, 'imaging_psf.fits')

### Deconvolve using clean

In [ ]:
parameters={'niter':100, 'threshold':0.001, 'fracthresh':0.01}
comp, residual = deconvolve_cube(dirty, psf, parameters)

# Show the results

fig=show_image(comp)
fig=show_image(residual)

In [ ]:
### Predict the visibility of the model

In [ ]:
vtmodel = create_visibility(vlaa, times, frequency, weight=1.0, phasecentre=phasecentre)
vtmodel=predict_2d(vtmodel, comp, params=parameters)

In [ ]:
### Now we will plot the original visibility and the residual visibility.

In [ ]:
uvdist=numpy.sqrt(vt.data['uvw'][:,0]**2+vt.data['uvw'][:,1]**2)
plt.clf()
plt.plot(uvdist, numpy.abs(vt.data['vis'][:,0,0]), '.', color='b')
plt.plot(uvdist, numpy.abs(vt.data['vis'][:,0,0]-vtmodel.data['vis'][:,0,0]), '.', color='r')
plt.xlabel('uvdist')
plt.ylabel('Amp Visibility')
plt.show()